### CNN on CIFR Assignment:

1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use Dense Layers (also called fully connected layers), or DropOut.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [0]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
# from tensorflow import keras

# from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
# import tensorflow as tf
#tf.config.gpu.set_per_process_memory_fraction(0.75)
#tf.config.gpu.set_per_process_memory_growth(True)
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

# Create a session with the above options specified.
# k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 85
l = 6
num_filter = 35
compression = 1
dropout_rate = 0.2

In [4]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
X_train.shape

(50000, 32, 32, 3)

In [0]:
X_test.shape

(10000, 32, 32, 3)

In [0]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Block
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(3,3))(relu)
    #flat = layers.Flatten()(AvgPooling)
    #a=tf.keras.layers.Reshape((None,312,1), input_shape=(2,2,78))
    #model=tf.reshape(flat,[None,312,1])
    output = layers.Conv2D(10,(1,1),strides=(1,1),activation='softmax',padding='valid')(AvgPooling)
    flat = layers.Flatten()(output)

    return flat

In [0]:
#https://www.kaggle.com/genesis16/densenet-93-accuracy
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 35)   945         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_52 (BatchNo (None, 32, 32, 35)   140         conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_52 (Activation)      (None, 32, 32, 35)   0           batch_normalization_52[0][0]     
______________________________________________________________________________________________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
from keras.preprocessing.image import ImageDataGenerator
generator = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.125,
                               height_shift_range=0.125,
                               horizontal_flip=True,
                               fill_mode='nearest',
                               zoom_range=0.10)

generator.fit(X_train)

In [22]:
#https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
model.fit_generator(generator.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=(len(X_train)//batch_size)*5, epochs=20,
                    validation_data=(X_test, y_test))

Epoch 1/20
3905/3905 [==============================] - 410s 105ms/step - loss: 0.4277 - acc: 0.8520 - val_loss: 0.8166 - val_acc: 0.7626
Epoch 2/20
3905/3905 [==============================] - 406s 104ms/step - loss: 0.3864 - acc: 0.8654 - val_loss: 0.5648 - val_acc: 0.8379
Epoch 3/20
3905/3905 [==============================] - 409s 105ms/step - loss: 0.3513 - acc: 0.8778 - val_loss: 0.4767 - val_acc: 0.8535
Epoch 4/20
3905/3905 [==============================] - 409s 105ms/step - loss: 0.3245 - acc: 0.8873 - val_loss: 0.4529 - val_acc: 0.8631
Epoch 5/20
3905/3905 [==============================] - 407s 104ms/step - loss: 0.3051 - acc: 0.8936 - val_loss: 0.3657 - val_acc: 0.8901
Epoch 6/20
3905/3905 [==============================] - 406s 104ms/step - loss: 0.2859 - acc: 0.8998 - val_loss: 0.4397 - val_acc: 0.8722
Epoch 7/20
3905/3905 [==============================] - 407s 104ms/step - loss: 0.2689 - acc: 0.9054 - val_loss: 0.4023 - val_acc: 0.8840
Epoch 8/20
3905/3905 [============

In [23]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 4s 444us/sample - loss: 0.3447 - acc: 0.9071
Test loss: 0.3446852895885706
Test accuracy: 0.9071


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


#Conclusions:

1- Used the given densenet to build the architecture of the model.

2- Used Image Augmentation techniques to make the model robust.

3- Used adam optimizer to optimize the loss.

4- Run the code for 20 epochs.

5- Test Loss=0.344

6- Test accuracy=90.71%